In [9]:
import numpy as np 
import pandas as pd
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests # library to handle requests
# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library
import seaborn as sns
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [12]:
toronto_data_w_gsptl = pd.read_csv('toronto_data_2.csv')

In [13]:
CLIENT_ID = 'UP1DLMNZTHCTJVGIJCZYTRNO3BHYKOGZ3GPVKRMN5KJSX5BS' # your Foursquare ID
CLIENT_SECRET = 'JGQW0RQJUJ0PSFRNMDXMG5SMWY1U3OBF5CIANCUKVTHLMB40' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: UP1DLMNZTHCTJVGIJCZYTRNO3BHYKOGZ3GPVKRMN5KJSX5BS
CLIENT_SECRET:JGQW0RQJUJ0PSFRNMDXMG5SMWY1U3OBF5CIANCUKVTHLMB40


In [14]:
toronto_data_w_gsptl.drop(columns = 'Postcode',inplace=True)
print(print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto_data_w_gsptl['Borough'].unique()),
        toronto_data_w_gsptl.shape[0])))
toronto_data_w_gsptl.head()

The dataframe has 11 boroughs and 103 neighborhoods.
None


,Borough,Neighbourhood,Latitude,Longitude
0,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,Scarborough,Woburn,43.770992,-79.216917
4,Scarborough,Cedarbrae,43.773136,-79.239476


In [15]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[toronto_data_w_gsptl.Latitude.mean(),
                                   toronto_data_w_gsptl.Longitude.mean()], zoom_start=10)

# add markers to map
iterator = zip(toronto_data_w_gsptl['Latitude'], toronto_data_w_gsptl['Longitude'],
                toronto_data_w_gsptl['Borough'], toronto_data_w_gsptl['Neighbourhood'])
for lat, lng, borough, neighborhood in iterator:
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Explore Neighborhoods in Toronto

In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        while True:
            try:
                results = requests.get(url).json()["response"]['groups'][0]['items']
                break
            except:
                continue
            
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
# toronto_venues = getNearbyVenues(names=toronto_data_w_gsptl['Neighbourhood'],
#                                    latitudes=toronto_data_w_gsptl['Latitude'],
#                                    longitudes=toronto_data_w_gsptl['Longitude']
#                                   )
##Checkpoint
# toronto_venues.to_csv('toronto_venues.csv', index=False)

In [19]:
toronto_venues = pd.read_csv('toronto_venues.csv')

In [21]:
print(toronto_venues.shape)
toronto_venues.head()

(38, 16)


,0,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Trail,Pub,Wine Bar,Cuban Restaurant,Falafel Restaurant,Ethiopian Restaurant,Eastern European Restaurant,Dog Run,Discount Store
1,2,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Ice Cream Shop,Italian Restaurant,Yoga Studio,Bookstore,Restaurant,Diner,Spa,Dessert Shop,Juice Bar
2,3,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0,Park,Coffee Shop,Ice Cream Shop,Liquor Store,Light Rail Station,Burger Joint,Burrito Place,Fast Food Restaurant,Fish & Chips Shop,Italian Restaurant
3,4,East Toronto,Studio District,43.659526,-79.340923,0,Cafe,Coffee Shop,American Restaurant,Bakery,Italian Restaurant,Comfort Food Restaurant,Chinese Restaurant,Seafood Restaurant,Latin American Restaurant,Bookstore
4,5,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Bus Line,Park,Swim School,Cuban Restaurant,Ethiopian Restaurant,Eastern European Restaurant,Dog Run,Discount Store,Diner,Dessert Shop


In [22]:
venue_counts = toronto_venues.groupby('Neighborhood').Venue.count()
venue_counts

AttributeError: 'DataFrameGroupBy' object has no attribute 'Venue'